# Llama 3.2 Vision Multi-Turn Flat Debit Extractor - CBA Dynamic

- This notebook demonstrates multi-turn conversational extraction using Llama 3.2 Vision.
- Uses the `chat_with_mllm` pattern to handle multiple turns of conversation with Flat Bank Statement images.
- **Dynamic column references**: Uses actual column names from Turn 0 instead of hardcoded values.
- **Position-based mapping**: Column roles determined by position (1=date, 2=description, 3=debit, 4=credit, 5=balance)

**Reference**: [Chat with Your Images Using Multimodal LLMs](https://medium.com/data-science/chat-with-your-images-using-multimodal-llms-60af003e8bfa)

## Imports

In [ ]:
# Add project root to path for common/ imports
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import random
import numpy as np
import torch
from PIL import Image
from transformers import AutoProcessor, MllamaForConditionalGeneration
from transformers.image_utils import load_image
from tqdm.notebook import tqdm
from IPython.display import display, Markdown
import re

## Pre-emptive Memory Cleanup

Optional GPU memory cleanup to prevent OOM errors when switching between models.

In [ ]:
# Optional: Pre-emptive memory cleanup (useful when switching models)
try:
    from common.gpu_optimization import emergency_cleanup
    print("🧹 Clearing GPU memory...")
    emergency_cleanup(verbose=False)
    print("✅ Memory cleanup complete")
except ImportError:
    print("⚠️ GPU optimization module not available - skipping cleanup")

## Set Random Seed for Reproducibility

In [ ]:
from common.reproducibility import set_seed
set_seed(42)

## Load Model

In [ ]:
model_id = "/home/jovyan/shared_PTM/Llama-3.2-11B-Vision-Instruct"

print("🔧 Loading Llama-3.2-Vision model...")

from common.llama_model_loader_robust import load_llama_model_robust

model, processor = load_llama_model_robust(
    model_path=model_id,
    use_quantization=False,
    device_map='auto',
    max_new_tokens=2000,
    torch_dtype='bfloat16',
    low_cpu_mem_usage=True,
    verbose=True
)

# Add tie_weights() call
try:
    model.tie_weights()
    print("✅ Model weights tied successfully")
except Exception as e:
    print(f"⚠️ tie_weights() warning: {e}")

print("✅ Model loaded successfully!")

## Optional: Manual Memory Cleanup

Run this cell if you experience memory issues during the conversation. Not needed for normal operation on H200.

In [ ]:
# # Optional: Run this cell if you experience memory issues during conversation
# import gc

# print("🧹 Manual memory cleanup...")
# gc.collect()
# torch.cuda.empty_cache()

# # Show memory status
# if torch.cuda.is_available():
#     for i in range(torch.cuda.device_count()):
#         allocated = torch.cuda.memory_allocated(i) / 1e9
#         reserved = torch.cuda.memory_reserved(i) / 1e9
#         print(f"   GPU {i}: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
# print("✅ Cleanup complete")

## Define chat_with_mllm Function

This function encapsulates the multi-turn conversation pattern.

In [ ]:
from common.llama_multiturn_chat import chat_with_mllm

print("✅ chat_with_mllm function imported from common.llama_multiturn_chat")

## Dynamic Column Mapping Functions

These functions parse Turn 0 output and generate prompts with actual column names.

In [ ]:
from common.header_mapping import parse_5_column_headers


def generate_dynamic_prompts(column_map):
    """Generate conversation prompts with actual column names from Turn 0.
    
    Args:
        column_map: dict from parse_5_column_headers() with keys: date, description, debit, credit, balance
    
    Returns:
        dict: CONVERSATION_PROMPTS dictionary with dynamic column references
    """
    return {
        "turn_0_identify_headers": """Look at the bank statement image.

Find the transaction table where transactions are listed.

At the top of this table, there is a row of column headers.

Your task: List ALL the column headers from this table, starting from the left edge and moving to the right edge.

IMPORTANT: The leftmost column is usually "Date" or "Date of Transaction". Start there and read every header moving right.

Include EVERY header you see. Do not skip any.

Output Format:
- TRANSACTION TABLE HEADERS (reading left to right):
- [Write ONLY the headers you actually see in the image, separated by commas]
- If no table headers visible, write: NO_HEADERS""",
        
        "turn_1_initial_extraction": f"""Extract the transaction table from this Australian bank statement in markdown format.

CRITICAL FORMATTING RULES:

1. DATE FORMAT:
   - Australian bank statements use "DD Mon YYYY" or "DD Mon" format
   - Example formats: "15 Mar 2024", "15 Mar", "15 March"
   - Each row has ONE date entry in the "{column_map['date']}" column (leftmost)

2. TRANSACTION DESCRIPTIONS:
   - Some transaction descriptions span multiple visual lines
   - You MUST combine all lines for a single transaction into ONE table row
   - Example: If you see:
     Line 1: "SUPERMARKET PAYMENT"
     Line 2: "Card xx1234"
     Line 3: "Value Date: 20/03/2024"
   - Combine as: "SUPERMARKET PAYMENT Card xx1234 Value Date: 20/03/2024"

3. COLUMN STRUCTURE:
   - The table has these columns: {column_map['date']} | {column_map['description']} | {column_map['debit']} | {column_map['credit']} | {column_map['balance']}
   - Maintain proper markdown table alignment with | separators
   - Empty cells should be represented with nothing between pipes

4. CR NOTATION (if applicable):
   - "{column_map['balance']}" column may show "CR" suffix (e.g., "$1,234.56 CR")
   - "{column_map['credit']}" column typically does NOT show "CR" suffix
   - Preserve "CR" notation exactly as shown in the image

Output the complete transaction table in markdown format.""",
        
        "turn_2_select_columns": f"""STEP 2: From the table you extracted in STEP 1, extract only the "{column_map['date']} | {column_map['description']} | {column_map['debit']}" columns""",
        
        "turn_3_extract_debits": f"""STEP 3: From the table you extracted in STEP 2, remove any row NOT showing an amount (i.e. having an empty cell) in the "{column_map['debit']}" column.""",
        
        "turn_4_date_range": f"""Extract earliest date and the latest date from the "{column_map['date']}" column. Express your answer in the format "STATEMENT_DATE_RANGE: dd/mm/yyyy - dd/mm/yyyy" """
    }

print("✅ parse_5_column_headers imported from common.header_mapping")
print("✅ generate_dynamic_prompts function defined")

## Static Turn 0 Prompt

Turn 0 prompt is always static (used to identify headers)

In [ ]:
TURN_0_PROMPT = """Look at the bank statement image.

Find the transaction table where transactions are listed.

At the top of this table, there is a row of column headers.

Your task: List ALL the column headers from this table, starting from the left edge and moving to the right edge.

IMPORTANT: The leftmost column is usually "Date" or "Date of Transaction". Start there and read every header moving right.

Include EVERY header you see. Do not skip any.

Output Format:
- TRANSACTION TABLE HEADERS (reading left to right):
- [Write ONLY the headers you actually see in the image, separated by commas]
- If no table headers visible, write: NO_HEADERS"""

print("✅ Turn 0 prompt defined")

# Select Image

In [ ]:
# Image path
imageName = "/home/jovyan/_LMM_POC/evaluation_data/image_003.png"


In [ ]:
# OPTIONAL: Uncomment to enable preprocessing
# from common.image_preprocessing import enhance_for_llama, preprocess_statement_for_llama, enhance_statement_quality
# from PIL import Image
# import tempfile

# # Choose ONE preprocessing approach:

# # Option 1: Light enhancement (recommended for high-quality scans)
# # preprocessed_img = enhance_statement_quality(imageName)

# # Option 2: Moderate enhancement (upscaling + sharpness + contrast)
# # preprocessed_img = enhance_for_llama(imageName)

# # Option 3: Aggressive preprocessing (denoise + binarize + remove lines)
# # preprocessed_img = preprocess_statement_for_llama(imageName)

# # Save preprocessed image to temporary file and update imageName
# # with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmp:
# #     preprocessed_img.save(tmp.name)
# #     imageName = tmp.name
# #     print(f"✅ Using preprocessed image: {imageName}")
# #     display(preprocessed_img)

print("⏭️  Skipping preprocessing - using original image")

## OPTIONAL: Image Preprocessing

**Experimental:** Test whether preprocessing improves extraction accuracy.

Available preprocessing functions:
- `enhance_for_llama()` - Upscale, sharpen, increase contrast
- `preprocess_statement_for_llama()` - Denoise, binarize, remove table lines
- `enhance_statement_quality()` - Moderate enhancement for bank statements

**Note:** Modern VLMs are trained on natural images. Preprocessing may help with low-quality scans but could hurt performance on high-quality images. Test both approaches to see what works best for your data.

## Turn 0: Identify Table Headers

First, identify all column headers in the transaction table.

In [ ]:
# Initialize conversation
messages = []
images = []

print("📸 Processing bank statement image...")
print(f"📝 Using prompt: turn_0_identify_headers")
print("\n" + "=" * 60)
print("PROMPT:")
print("=" * 60)
print(TURN_0_PROMPT)
print("=" * 60 + "\n")

response0, messages, images = chat_with_mllm(
    model, 
    processor, 
    TURN_0_PROMPT,
    images_path=[imageName],
    do_sample=False,
    max_new_tokens=500,
    show_image=True,
    messages=messages,
    images=images
)

print("\n" + "=" * 60)
print("TURN 0 - IDENTIFY TABLE HEADERS:")
print("=" * 60)
print(response0)
print("=" * 60)

## Parse Column Headers and Generate Dynamic Prompts

Extract actual column names from Turn 0 and create prompts that reference them.

In [ ]:
# Parse Turn 0 response to extract column names
column_map = parse_5_column_headers(response0)

print("\n" + "=" * 60)
print("COLUMN MAPPING (Position-Based):")
print("=" * 60)
print(f"Position 1 (Date):        '{column_map['date']}'")
print(f"Position 2 (Description): '{column_map['description']}'")
print(f"Position 3 (Debit):       '{column_map['debit']}'")
print(f"Position 4 (Credit):      '{column_map['credit']}'")
print(f"Position 5 (Balance):     '{column_map['balance']}'")
print("=" * 60)

# Generate dynamic prompts using actual column names
CONVERSATION_PROMPTS = generate_dynamic_prompts(column_map)
print("\n✅ Dynamic prompts generated with actual column names")

## Turn 1: Extract Transaction Table

Extract the complete transaction table in markdown format using dynamic column names.

In [ ]:
print(f"📝 Using prompt: turn_1_initial_extraction")
print("\n" + "=" * 60)
print("PROMPT:")
print("=" * 60)
print(CONVERSATION_PROMPTS["turn_1_initial_extraction"])
print("=" * 60 + "\n")

response1, messages, images = chat_with_mllm(
    model, 
    processor, 
    CONVERSATION_PROMPTS["turn_1_initial_extraction"],
    messages=messages,
    images=images,
    do_sample=False,
    max_new_tokens=3000
)

display(Markdown(response1))

# Save initial extraction
Path("llama_debit_extractor_initial_dynamic.txt").write_text(response1)
print("\n✅ Initial extraction saved to llama_debit_extractor_initial_dynamic.txt")

## Turn 2: Select Columns

In [ ]:
print(f"📝 Using prompt: turn_2_select_columns")
print("\n" + "=" * 60)
print("PROMPT:")
print("=" * 60)
print(CONVERSATION_PROMPTS["turn_2_select_columns"])
print("=" * 60 + "\n")

response2, messages, images = chat_with_mllm(
    model, processor,
    CONVERSATION_PROMPTS["turn_2_select_columns"],
    messages=messages, 
    images=images,
    max_new_tokens=2000
)

display(Markdown(response2))

## Turn 3: Extract Debit/Withdrawal Amounts Only

In [ ]:
print(f"📝 Using prompt: turn_3_extract_debits")
print("\n" + "=" * 60)
print("PROMPT:")
print("=" * 60)
print(CONVERSATION_PROMPTS["turn_3_extract_debits"])
print("=" * 60 + "\n")

response3, messages, images = chat_with_mllm(
    model, processor,
    CONVERSATION_PROMPTS["turn_3_extract_debits"],
    messages=messages,
    images=images,
    max_new_tokens=2000
)

display(Markdown(response3))

# Save debit amounts
Path("llama_debit_amounts_dynamic.txt").write_text(response3)
print("\n✅ Debit amounts saved to llama_debit_amounts_dynamic.txt")

## Turn 4: Date Range

In [ ]:
print(f"📝 Using prompt: turn_4_date_range")
print("\n" + "=" * 60)
print("PROMPT:")
print("=" * 60)
print(CONVERSATION_PROMPTS["turn_4_date_range"])
print("=" * 60 + "\n")

response4, messages, images = chat_with_mllm(
    model, processor,
    CONVERSATION_PROMPTS["turn_4_date_range"],
    messages=messages,
    images=images,
    max_new_tokens=50
)

print("\n" + "=" * 60)
print("TURN 4 - DATE RANGE:")
print("=" * 60)
print(response4)
print("=" * 60)

## Debug: View Conversation Structure

In [ ]:
print("🔍 Current conversation structure:")
print("=" * 60)
for i, msg in enumerate(messages, 1):
    print(f"\nMessage {i} ({msg['role']}):")
    for content in msg['content']:
        if content['type'] == 'text':
            preview = content['text'][:100] + "..." if len(content['text']) > 100 else content['text']
            print(f"  [text]: {preview}")
        else:
            print(f"  [{content['type']}]")
print("=" * 60)
print(f"\n📊 Total messages: {len(messages)}")
print(f"📊 User messages: {sum(1 for m in messages if m['role'] == 'user')}")
print(f"📊 Assistant messages: {sum(1 for m in messages if m['role'] == 'assistant')}")

## Save Full Conversation

In [ ]:
# Save the entire conversation to a file
output_path = Path("llama_multiturn_debit_conversation_dynamic.txt")

with output_path.open("w", encoding="utf-8") as text_file:
    text_file.write("=" * 60 + "\n")
    text_file.write("MULTI-TURN DEBIT EXTRACTION CONVERSATION (DYNAMIC)\n")
    text_file.write("Llama-3.2-Vision-11B\n")
    text_file.write("=" * 60 + "\n\n")
    
    # Write column mapping
    text_file.write("COLUMN MAPPING:\n")
    text_file.write(f"  Date column:        {column_map['date']}\n")
    text_file.write(f"  Description column: {column_map['description']}\n")
    text_file.write(f"  Debit column:       {column_map['debit']}\n")
    text_file.write(f"  Credit column:      {column_map['credit']}\n")
    text_file.write(f"  Balance column:     {column_map['balance']}\n")
    text_file.write("\n" + "=" * 60 + "\n\n")
    
    for i, msg in enumerate(messages, 1):
        role = msg["role"].upper()
        text_file.write(f"\n{'-' * 60}\n")
        text_file.write(f"MESSAGE {i} - {role}\n")
        text_file.write(f"{'-' * 60}\n\n")
        
        for content in msg["content"]:
            if content["type"] == "text":
                text_file.write(content["text"] + "\n")
            elif content["type"] == "image":
                text_file.write("[IMAGE]\n")
    
    text_file.write("\n" + "=" * 60 + "\n")
    text_file.write(f"Total messages: {len(messages)}\n")
    text_file.write("=" * 60 + "\n")

print(f"✅ Full conversation saved to: {output_path}")
print(f"📊 File size: {output_path.stat().st_size} bytes")
print(f"💬 Total messages in conversation: {len(messages)}")